# 井字棋-博弈搜索

*  游戏规则:双方轮流放子,当某一方的三个子连成一线(行,列,对角)时,该方获胜。

![title](other_data/01.jpg)

# 程序设计思想
1. 首先选择先手的玩家，如果选择“X”表示“玩家”先手，如果选择“O”表示“电脑”先手，如果输入错误就会默认“玩家”先手。
2. 我们通过（0-8）的数字表示棋盘上的位置，“玩家”和“电脑”会轮流下棋，如果一方获得胜利，或者平局的时候（棋盘上没有空位置）就结束。当轮到“玩家”下棋，就让玩家自己输入（0-8）的数字，但是要求不能输入棋盘上已经有棋子的位置，否则就让玩家重新输入，如果轮到“电脑”下棋，那么就让“电脑”采用极大极小值算法自动选择最适合下棋的位置。
    * 用以下的9个数字来表示棋盘的位置:
    * 0  1  2
    * 3  4  5
    * 6  7  8

3. 在该程序中，我们会用-1，1来表示具体的玩家，0来表示空位置，这样就会方便我们计算胜利或失败的情况。

![title](other_data/02.jpg)

* 我们用-1（绿色方块）来表示“玩家”，1（红色方块）来表示“电脑”，0（灰色方块）表示空位置。

## 1.初始化参数

In [ ]:
import random

# 用一维列表表示棋盘:
SLOTS = (0, 1, 2, 3, 4, 5, 6, 7, 8)

# -1表示X玩家 0表示空位 1表示O玩家,在游戏时就会生成类似[-1, 1, -1, 0, 1, 0, 0, 0, 0]的数组
X_token = -1
None_token = 0
O_token = 1

# 设定获胜的组合方式(横、竖、斜)
WINNING_TRIADS = ((0, 1, 2), (3, 4, 5), (6, 7, 8),
                  (0, 3, 6), (1, 4, 7),(2, 5, 8),
                  (0, 4, 8), (2, 4, 6))

# 三种结果
result = ('平局', '胜利', '失败')

## 2.判断棋盘上是否还有空位
* 如果棋盘中有 0 存在，那么就有空位置，返回True,否则就返回False。

In [ ]:
def legal_move_left(board):
    for slot in SLOTS:
        if board[slot] == None_token:
            return True
    return False

## 3.判断棋盘上是否有获胜者
* 判断局面的胜者,如果“玩家”获胜返回-1，如果“电脑”获胜返回1，平局或者未结束返回0。

In [ ]:
def winner(board):
    for triad in WINNING_TRIADS:
        triad_sum = board[triad[0]] + board[triad[1]] + board[triad[2]]
        # 如果在获胜组合中每一个元素都等于1，那么相加就等于3，也就是说在获胜组合中都是“电脑”，因此“电脑”就赢了，返回 1
        if triad_sum == 3:
            return 1
         # 如果在获胜组合中每一个元素都等于-1，那么相加就等于-3，也就是说在获胜组合中都是“玩家”，因此“玩家”就赢了，返回 -1
        elif triad_sum == -3:
            return -1
    # 其它情况都返回 0 。
    return 0

## 极大极小值算法计算当前位置分数

* 在决策树中，轮到我方决策层时，我们总希望做出得分最高的决策（得分以我方标准来算）；而在敌方决策层时，我们假定敌方总能够做出得分最小的决策（我方得分最小便是相应敌方得分最高）。所以在博弈树中，每一层所要追求的结果，在极大分数和极小分数中不断交替，故称之为极大极小搜索。

* 我们会搜索“玩家”和“电脑”对弈的所有情况，每一种情况都有一个棋盘状态，因此最后肯定有结果（输，赢，平局），我们规定，输=-1，赢=1，平局=0。

* 我们观察下图第一排，第一排由“电脑”（O）下棋得到，因此“电脑”会选择最大的状态值，第二排由“玩家”（X）下棋得到，因此“电脑”会选择当前所有局面最小的状态值，以此类推，第三排“电脑”选择最大值，第四排“电脑”选择最小值。

![title](other_data/03.jpg)

* 这里我们可以通过剪枝的方式来剪掉一些多余的分支，也就是剪掉状态值大于等于2，或者小于等于-2的情况，因此我们用两个参数，alpha=-2, beta=2作为该算法的剪枝参数。

In [ ]:

def alpha_beta_valuation(board, player, next_player, alpha, beta):
    # 判断是否有玩家获胜
    wnnr = winner(board)
    if wnnr != None_token:
        # 有玩家获胜
        return wnnr
    # 没有空位,平局，返回 0
    elif not legal_move_left(board):
        return 0
    
    # 遍历所有可以下棋的位置
    for move in SLOTS:
        # 只能下在空位置上。
        if board[move] == None_token:
            # 默认由“电脑”开始下棋，因此这次就由“玩家”下棋并更新状态值
            board[move] = player
            # “玩家”下完棋之后交换玩家，由“电脑”下棋，通过递归的方法计算状态值。
            val = alpha_beta_valuation(board, next_player, player, alpha, beta)
            # 把空位置状态还原
            board[move] = None_token
            # 如果是“电脑”下棋,“电脑”就会选择当前状态下的最大评估值。
            if player == O_token:
                # 对于“电脑”来说，只要有比最小值alpha要大的话，就会选择更大的状态值val。
                if val > alpha:
                    alpha = val
                # 对结果进行剪枝，只要有比最大值beta大的，就选择我们限定的最大值beta
                if alpha >= beta:
                    return beta
                
             # 如果是“玩家”下棋，对于“电脑”来说就会选择当前状态下的最小评估值。
            else: 
                # 对于“电脑”来说，只要有比最大值beta小的，就选择更小的状态值val
                if val < beta:
                    beta = val
                 # 对结果进行剪枝，只要有比最小值alpha小的，就选择我们限定的最小值alpha
                if beta <= alpha:
                    return alpha
    # 如果当前玩家是“电脑”，就找出对“电脑”来说最坏的情况，一共三种情况，输赢和平局
    if player == O_token:
        retval = alpha
    # 如果当前玩家是“玩家”，就找出对“电脑”来说最好的情况
    else:
        retval = beta
    return retval

## 决定电脑下棋的位置
* 创建空列表接收通过极大极小值算法计算的最大分值的位置信息，如果有多个相同最大分值的位置信息就随机挑选一个位置。

In [ ]:
def determine_move(board):
    
    min_val = -2
    my_moves = []
    print("正在思考")
    # 遍历所有棋盘的每一个位置
    for move in SLOTS:
        # 判断当前move是否为0
        if board[move] == None_token:
            # 默认让“电脑”先下棋，下一次就由“玩家”X_token下棋
            board[move] = O_token
            # 当前的状态值。
            val = alpha_beta_valuation(board, X_token, O_token, -2, 2)
            # 恢复“电脑”占用的空位置。
            board[move] = None_token
            # 根据极大极小值val判断“电脑”的输赢。
            print("电脑如果下在", move, ",将导致", result[val])
            # 只要有比设定的最小状态值更大那么就当记录最大的这个状态值，并记录走的哪一步。
            if val > min_val:
                min_val = val
                my_moves = [move]
            # 如果有当前状态值与之前最大的状态值相等，就记录当前走的哪一步。
            if val == min_val:
                my_moves.append(move)
    # 如果电脑可以有多个最大并且相等的状态值，那么就随机选择一步。
    return random.choice(my_moves)

## 主函数
* 轮流让“玩家”与“电脑”进行博弈，一直循环直到有一方获胜或者棋盘上没有空位置为止。

In [ ]:
HUMAN = 1
COMPUTER = 0
import show_box
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import Button, GridBox, Layout, ButtonStyle

"""程序入口,先决定谁是先手方,再开始下棋"""
next_move = HUMAN
opt = input("请选择先手方，输入X表示玩家先手，输入O表示电脑先手：")
if opt == "X":
    next_move = HUMAN
elif opt == "O":
    next_move = COMPUTER
else:
    print("输入有误，默认玩家先手")

def display_box(board):
    box_values = show_box.plot(board).values()
    display(GridBox(children=list(box_values),layout = Layout(
            width='80%',
            grid_template_columns='40px 40px 40px',
            grid_template_rows='40px 40px 40px',
            grid_gap='1px')))
    
    
    
    
# 初始化空棋盘
board = [None_token for i in range(9)]
# 开始下棋
# 一直循环直到有一方获胜或者棋盘上没有空位置为止。
while legal_move_left(board) and winner(board) == None_token:
    # 将棋局显示出来
    print(board)
    display_box(board)
    # 如果轮到玩家下棋，并且棋盘上有空格并且对方没有获得胜利的时候就让玩家选择下棋位置。
    if next_move == HUMAN and legal_move_left(board):
        try:
            humanmv = int(input("请输入你要落子的位置(0-8)："))
            if board[humanmv] != None_token:
                continue
            board[humanmv] = X_token
            next_move = COMPUTER
        except:
            print("输入有误，请重试")
            continue
    # 如果轮到电脑下棋，并且棋盘上有空格并且玩家没有获得胜利的时候就让电脑选择下棋位置。
    if next_move == COMPUTER and legal_move_left(board):
        mymv = determine_move(board)
        print("Computer最终决定下在", mymv)
        board[mymv] = O_token
        next_move = HUMAN

# 输出结果
display_box(board)
print(["平局", "电脑赢了", "你赢了"][winner(board)])